<a href="https://colab.research.google.com/github/Mudassir-beep/AI-Disease-diagnostics/blob/main/medica_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ✅ 0. Upload JSON dataset file
from google.colab import files
uploaded = files.upload()  # Upload your medical_qa_dataset.json file here

# ✅ 1. Load and Preview JSON
import pandas as pd

df = pd.read_json("medical_qa_dataset.json")
print("✅ Preview:")
print(df.head())

# ✅ 2. Save as CSV (optional but required for current code)
df.to_csv("medical_qa_dataset.csv", index=False)

# ✅ 3. Import Libraries
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

# ✅ 4. Load Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama",
    max_seq_length=512,
    dtype=None,
    load_in_4bit=True,
)

# ✅ 5. Apply QLoRA via PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

# ✅ 6. Load & Format Dataset
dataset = load_dataset("csv", data_files={"train": "medical_qa_dataset.csv"})

def format(example):
    return {
        "text": f"### Question:\n{example['question']}\n\n### Answer:\n{example['answer']}"
    }

dataset = dataset.map(format)
tokenized = dataset.map(lambda e: tokenizer(e["text"]), batched=True)
tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# ✅ 7. Training Arguments
training_args = TrainingArguments(
    output_dir="tinyllama_medbot_finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    max_steps=50,  # adjust as needed
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_strategy="epoch",
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    push_to_hub=False,
    report_to="none",
)

# ✅ 8. Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# ✅ 9. Trainer Setup
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ 10. Train
trainer.train()

# ✅ 11. Save Final Model
model.save_pretrained("tinyllama_medbot_finetuned", safe_serialization=True)
tokenizer.save_pretrained("tinyllama_medbot_finetuned")


Saving medical_qa_dataset.json to medical_qa_dataset (1).json
✅ Preview:
                               question  \
0      What are the causes of diabetes?   
1    What are the symptoms of diabetes?   
2   What are the treatment of diabetes?   
3  What are the prevention of diabetes?   
4        What are the diet of diabetes?   

                                              answer  
0  The causes of diabetes vary but often include ...  
1  The symptoms of diabetes vary but often includ...  
2  The treatment of diabetes vary but often inclu...  
3  The prevention of diabetes vary but often incl...  
4  The diet of diabetes vary but often include co...  
==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

/tmp/ipython-input-6-3002201639.py:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 83 | Num Epochs = 5 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 12,615,680 of 1,112,664,064 (1.13% trained)


Step,Training Loss
5,3.178800
10,3.099000
15,3.247200
20,3.100600
25,3.108300
30,3.193700
35,3.169600
40,3.100600
45,3.167700
50,3.171000


('tinyllama_medbot_finetuned/tokenizer_config.json',
 'tinyllama_medbot_finetuned/special_tokens_map.json',
 'tinyllama_medbot_finetuned/tokenizer.model',
 'tinyllama_medbot_finetuned/added_tokens.json',
 'tinyllama_medbot_finetuned/tokenizer.json')

In [2]:
!pip install -q streamlit langchain faiss-cpu sentence-transformers
!npm install -g localtunnel
!pip install -U langchain



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 834.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
# ✅ 1. Imports
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch

# ✅ 2. Load fine-tuned model + tokenizer
@st.cache_resource
def load_model():
    model = AutoModelForCausalLM.from_pretrained(
        "tinyllama_medbot_finetuned",  # this folder must be in your Colab or working dir
        torch_dtype=torch.float16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained("tinyllama_medbot_finetuned")
    return model, tokenizer

# ✅ 3. Load FAISS knowledge base
@st.cache_resource
def load_faiss():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.load_local(
        "faiss_medical_knowledge",  # make sure this folder exists in Colab
        embeddings,
        allow_dangerous_deserialization=True
    )
    return vectorstore

# ✅ 4. Get RAG pipeline using HuggingFacePipeline wrapper
def get_rag_chain(model, tokenizer, retriever):
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        repetition_penalty=1.1,
    )

    hf_llm = HuggingFacePipeline(pipeline=pipe)

    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        You are a helpful medical assistant. Use the provided context to answer the question.

        Context:
        {context}

        Question:
        {question}

        Answer:
        """,
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_template},
        return_source_documents=False,
    )
    return qa_chain

# ✅ 5. Streamlit UI
st.set_page_config(page_title="🧠 Medical RAG Chatbot", layout="centered")
st.title("🩺 Medical RAG Chatbot")
st.markdown("Ask any medical question based on fine-tuned TinyLlama + FAISS medical knowledge base.")

user_question = st.text_input("💬 Ask your question:")
submit = st.button("🔍 Get Answer")

# ✅ 6. Load everything & respond
if submit and user_question:
    with st.spinner("🔄 Loading model and knowledge base..."):
        model, tokenizer = load_model()
        retriever = load_faiss().as_retriever()
        qa_chain = get_rag_chain(model, tokenizer, retriever)

    with st.spinner("🤖 Generating answer..."):
        response = qa_chain.run(user_question)
        st.success("✅ Answer:")
        st.write(response)



ModuleNotFoundError: No module named 'langchain_community'